In [5]:
pip install tiktoken

  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ------------- -------------------------- 307.2/893.9 kB 2.1 MB/s eta 0:00:01
   ------------------- -------------------- 440.3/893.9 kB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 440.3/893.9 kB 1.6 MB/s eta 0:00:01
   ---------------------- ----------------- 512.0/893.9 kB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 573.4/893.9 kB 1.4 MB/s eta 0:00:01
   --------------------------- ------------ 624.6/893.9 kB 1.3 MB/s eta 0:00:01
   ------------------------------- -------- 706.6/893.9 kB 1.4 MB/s eta 0:00:01
   --------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import importlib
import tiktoken

print('titoken_verion:' , importlib.metadata.version('tiktoken'))

titoken_verion: 0.9.0


In [8]:
#Initializing the bpe from tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [9]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    " of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)


[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [10]:
string = tokenizer.decode(integers)

In [11]:
print(string)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [12]:
### Input-Target Pairs

In [15]:
with open ("the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()
encoded_text = tokenizer.encode(raw_text)
    
print("Text length:", len(encoded_text))

Text length: 5145


In [16]:
enc_sample = encoded_text[:100]

In [48]:
# So I want to create input-target pairs
# I'd create my content size, and this is... i.e the amount of tokens the model is to train to predict the next token
content_size = 5
x = enc_sample[:content_size]
y = enc_sample[1:content_size + 1]

In [49]:
print ("x:", x)
print ("y:", y)

x: [40, 367, 2885, 1464, 1807]
y: [367, 2885, 1464, 1807, 3619]


In [52]:
#processing the inputs along with the target
for i in range(1, content_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->" , desired)

[40] ----> 367
[40, 367] ----> 2885
[40, 367, 2885] ----> 1464
[40, 367, 2885, 1464] ----> 1807
[40, 367, 2885, 1464, 1807] ----> 3619


In [53]:
#Now I want to convert the token IDs back to text
for i in range(1, content_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->" , tokenizer.decode([desired]))


I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought
I HAD always thought ---->  Jack


In [56]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


c:\End-to-end_MLProject\llm-Tokenization\venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [57]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader


In [58]:
import torch

print("PyTorch version:", torch.__version__)

dataloader = create_dataloader_v1(
    raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


PyTorch version: 2.7.1+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [59]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [60]:
dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=8, 
    max_length=4, 
    stride=4, 
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
